In [1]:
#Author: Teo Lombardo

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px
from skimage import io, img_as_float, img_as_ubyte
from skimage.filters import gaussian
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.exposure import equalize_adapthist
import pickle

# Reading the input file
Inputs = pd.read_excel (r'Inputs_ToF_Images.xlsx')

path_data = (Inputs["Parameters' value"][1])
print("Results path =", path_data)

splitted_peaks = Inputs["Parameters' value"][2].split(",")
for i in range(len(splitted_peaks)):
    important_peaks = [x.replace(" ", "") for x in splitted_peaks]
print("Important ion fragment =", important_peaks)

norm_peak = str(Inputs["Parameters' value"][3])
if norm_peak == "nan":
    norm_peak = ""
print("Normalization peak =", norm_peak)

Results path = D:\Teo\ProGral\Prali\Data\ToF-SIMS\Analyses\Pre_lithiation_project\Pixel_normalization\Non cycled
Important ion fragment = ['LiOH-', 'LiO_2-', 'LiCO_3-', 'LiCO_2-', 'LiF_2-', 'Li_2F_3-', 'Li_4F_5-', 'Li_5F_6-', 'Li_3F_4-', 'LiPO_3-', 'LiPO_4-', 'LiPO_2-', 'LiPO-']
Normalization peak = C_2-


In [2]:
# Check previous
Samples_dir = [x[0] for x in os.walk(path_data)]
test_previous = path_data + "\\Box_plots_results_img"
previous = 0
if test_previous in Samples_dir:
    previous = 1

#previous = 0
print(previous)

0


In [3]:
Samples_dir

['D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled',
 'D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\00_Ref',
 "D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\0_5' - 20",
 "D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\0_5' - 40",
 "D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\0_5' - 60",
 'D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\1h - 20',
 'D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\1h - 40',
 'D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\Analyses\\Pre_lithiation_project\\Pixel_normalization\\Non cycled\\1h - 60',
 'D:\\Teo\\ProGral\\Prali\\Data\\ToF-SIMS\\A

In [4]:
dict_all = {}
normalization_dict = {}
dict_sums = {}
for directory in Samples_dir[1:]:
    if "Box_plots_results_img" not in directory:
        sample_name = directory.split("\\")[-1]
        #print(sample_name)
        dict_all[sample_name] = {}
        normalization_dict[sample_name] = {}
        measurements = os.listdir(directory)

In [ ]:
# Initializing and filling dictionaries (first run)

if previous == 0:
    dict_all = {}
    normalization_dict = {}
    dict_sums = {}
    for directory in Samples_dir[1:]:
        if "Box_plots_results_img" not in directory:
            sample_name = directory.split("\\")[-1]
            #print(sample_name)
            dict_all[sample_name] = {}
            normalization_dict[sample_name] = {}
            measurements = os.listdir(directory)

            for img_n in measurements:
                measure_name = img_n.split(" (")[0]
                peak_name = img_n.split(") ")[1].split("- ")[1].split(".txt")[0]
                #print(measure_name, peak_name)
                dict_all[sample_name][measure_name] = {}
                normalization_dict[sample_name][measure_name] = {}
            for img_n in measurements:
                measure_name = img_n.split(" (")[0]
                peak_name = img_n.split(") ")[1].split("- ")[1].split(".txt")[0]
                #print(measure_name, peak_name)
                dict_all[sample_name][measure_name][peak_name] = {}
                peak_dict = dict_all[sample_name][measure_name][peak_name]
                if peak_name == norm_peak:
                    normalization_dict[sample_name][measure_name][peak_name] = {}
                    Norm_to_fill = normalization_dict[sample_name][measure_name][peak_name]
                    
                path_img = directory + "\\" + img_n
                if peak_name != "total":
                    dict_sums[peak_name] = {}
                    dict_sums[peak_name]["Sample"] = []
                    dict_sums[peak_name]["Intensity"] = []
                    dict_sums[peak_name]["Normalized intensity"] = []
                    if len(norm_peak)>0:
                        dict_sums[peak_name]["Custom normalized intensity"] = []

                with open(path_img, 'r') as f:
                    lines = f.readlines()
                    f.close()

                count_line = 0
                for line in lines:
                    splitted_line = line.split(" ")
                    #print(splitted_line)
                    if splitted_line[0] != "#":
                        header = count_line+1
                        break
                    count_line+=1

                max_pixel = 0
                max_I = 0
                for line in lines[header:]:
                    splitted_line = line.split(" ")
                    pixel = float(splitted_line[0])
                    if pixel == 0:
                        max_pixel += 1
                    I = float(splitted_line[2])
                    if I > max_I:
                        max_I = I

                peak_dict["I"] = [[] for x in range(max_pixel)]
                peak_dict["In"] = [[] for x in range(max_pixel)]
                if peak_name == norm_peak:
                    Norm_to_fill["I"] = [[] for x in range(max_pixel)]
                for line in lines[header:]:
                    splitted_line = line.split(" ")
                    X = int(splitted_line[0]) 
                    Y = int(splitted_line[1]) 
                    I = float(splitted_line[2])
                    if max_I >0:
                        In = I/max_I
                    else: # To be taken in mind, it is weird that I need this
                        In = 0
                    peak_dict["I"][Y].append(I) 
                    peak_dict["In"][Y].append(In) 
                    if peak_name == norm_peak:
                        Norm_to_fill["I"][Y].append(I)
                peak_dict["I"] = np.array(peak_dict["I"])
                peak_dict["In"] = np.array(peak_dict["In"])
                if peak_name == norm_peak:
                    yi = 0
                    for y in Norm_to_fill["I"]:
                        xi = 0
                        for x in y:
                            if Norm_to_fill["I"][yi][xi] == 0:
                                Norm_to_fill["I"][yi][xi] = 0.5
                            xi+=1
                        yi+=1
                    Norm_to_fill["I"] = np.array(Norm_to_fill["I"])
                
    #pkl_filename = "dict_all_images.pkl"
    #with open(pkl_filename, 'wb') as file:
    #    pickle.dump(dict_all, file)

In [ ]:
# Initializing and filling normalization dictionary (not first run)
if previous == 1 and len(norm_peak)>0:
    print("Restarting")
    pkl_filename = path_data + "\\" + "dict_all_images.pkl"
    with open(pkl_filename, 'rb') as file:
        dict_all = pickle.load(file)
        
    normalization_dict = {}
    dict_sums = {}
    for directory in Samples_dir[1:]:
        if "Box_plots_results_img" not in directory:
            sample_name = directory.split("\\")[-1]
            normalization_dict[sample_name] = {}
            measurements = os.listdir(directory)

            for img_n in measurements:
                measure_name = img_n.split(" (")[0]
                peak_name = img_n.split(") ")[1].split("- ")[1].split(".txt")[0]
                normalization_dict[sample_name][measure_name] = {}
            for img_n in measurements:
                measure_name = img_n.split(" (")[0]
                peak_name = img_n.split(") ")[1].split("- ")[1].split(".txt")[0]
                if peak_name == norm_peak:
                    normalization_dict[sample_name][measure_name][peak_name] = {}
                    Norm_to_fill = normalization_dict[sample_name][measure_name][peak_name]
                    
                path_img = directory + "\\" + img_n
                if peak_name != "total":
                    dict_sums[peak_name] = {}
                    dict_sums[peak_name]["Sample"] = []
                    dict_sums[peak_name]["Intensity"] = []
                    dict_sums[peak_name]["Normalized intensity"] = []
                    if len(norm_peak)>0:
                        dict_sums[peak_name]["Custom normalized intensity"] = []

                if peak_name == norm_peak:
                    with open(path_img, 'r') as f:
                        lines = f.readlines()
                        f.close()

                if peak_name == norm_peak:
                    count_line = 0
                    for line in lines:
                        splitted_line = line.split(" ")
                        if splitted_line[0] != "#":
                            header = count_line+1
                            break
                        count_line+=1

                if peak_name == norm_peak:
                    max_pixel = 0
                    max_I = 0
                    for line in lines[header:]:
                        splitted_line = line.split(" ")
                        pixel = float(splitted_line[0])
                        if pixel == 0:
                            max_pixel += 1
                        I = float(splitted_line[2])
                        if I > max_I:
                            max_I = I

                if peak_name == norm_peak:
                    Norm_to_fill["I"] = [[] for x in range(max_pixel)]
                    for line in lines[header:]:
                        splitted_line = line.split(" ")
                        X = int(splitted_line[0]) 
                        Y = int(splitted_line[1]) 
                        I = float(splitted_line[2])
                        In = I/max_I
                        if peak_name == norm_peak:
                            Norm_to_fill["I"][Y].append(I)
                    if peak_name == norm_peak:
                        yi = 0
                        for y in Norm_to_fill["I"]:
                            xi = 0
                            for x in y:
                                if Norm_to_fill["I"][yi][xi] == 0:
                                    Norm_to_fill["I"][yi][xi] = 0.5
                                xi+=1
                            yi+=1
                        Norm_to_fill["I"] = np.array(Norm_to_fill["I"])

In [ ]:
# Sums and normalization(s)
for directory in Samples_dir[1:]:
    if "Box_plots_results_img" not in directory:
        sample_name = directory.split("\\")[-1]
        #dict_all_norm_t[sample_name] = {}
        measurements = os.listdir(directory)
        for img_n in measurements:
            measure_name = img_n.split(" (")[0]
            peak_name = img_n.split(") ")[1].split("- ")[1].split(".txt")[0]
            #for peak in dict_all[sample_name][measure_name]:
            if peak_name != "total":
                dict_sums[peak_name]["Sample"].append(sample_name)
                dict_sums[peak_name]["Intensity"].append(np.sum(dict_all[sample_name][measure_name][peak_name]["I"]))
                dict_all[sample_name][measure_name][peak_name]["I_norm_t"] = dict_all[sample_name][measure_name][peak_name]["I"]/dict_all[sample_name][measure_name]["total"]["I"]
                dict_all[sample_name][measure_name][peak_name]["I_norm_t"] = dict_all[sample_name][measure_name][peak_name]["I_norm_t"] / (max_pixel*max_pixel)
                if len(norm_peak)>0:
                    dict_all[sample_name][measure_name][peak_name]["Cust_norm"] = dict_all[sample_name][measure_name][peak_name]["I"]/normalization_dict[sample_name][measure_name][norm_peak]["I"]
                
                if peak_name == norm_peak and len(norm_peak)>0:
                    yi = 0
                    for y in dict_all[sample_name][measure_name][norm_peak]["Cust_norm"]:
                        xi = 0
                        for x in y:
                            if dict_all[sample_name][measure_name][norm_peak]["Cust_norm"][yi][xi] == 0:
                                dict_all[sample_name][measure_name][norm_peak]["Cust_norm"][yi][xi] = 1
                            xi+=1
                        yi+=1
                        
                dict_sums[peak_name]["Normalized intensity"].append(np.sum(dict_all[sample_name][measure_name][peak_name]["I_norm_t"]))
                if len(norm_peak)>0:
                    dict_sums[peak_name]["Custom normalized intensity"].append(np.sum(dict_all[sample_name][measure_name][peak_name]["Cust_norm"]))

In [ ]:
#Creating results' folder 
result_folder_name = "Box_plots_results_img"
directory_res = path_data + '\\' + result_folder_name
directory_res_tot = directory_res + "\\" + "Normalization by total counts"
directory_res_cust = directory_res + "\\" + "Customized normalization_" + str(norm_peak)
directory_res_tot_extra = directory_res_tot + '\\' + "Extra"
directory_res_cust_extra = directory_res_cust + '\\' + "Extra"

try: 
    os.makedirs(directory_res)
except:
    pass

try: 
    os.makedirs(directory_res_tot)
except:
    pass

if len(norm_peak)>0:
    try: 
        os.makedirs(directory_res_cust)
    except:
        pass
    
try: 
    os.makedirs(directory_res_tot_extra)
except:
    pass

if len(norm_peak)>0:
    try: 
        os.makedirs(directory_res_cust_extra)
    except:
        pass

In [ ]:
# Preparing the plottings
data_per_sample_N = {}
data_per_sample_CN = {}
averages_N = {}
#averages_N["Sample"] = []
averages_CN = {}
#averages_CN["Sample"] = []
reordering = 0
#ordered_sample_list = ["MK-220405-BASF-25", "MK-220328-BASF-100", "MK-220325-BASF-200", "MK-220323-BASF-300",
#                      "MK-220324-BASF-400", "MK-220412-BASF-500", "MK-220328-BASF-600", "MK-220324-BASF-700"]

In [ ]:
# Plotting for original and total ions normalization
for peak in dict_sums:
    df_to_plot_0 = pd.DataFrame.from_dict(dict_sums[peak])
    if reordering == 1:
        dict_r = {}
        dict_r["Sample"] = []
        dict_r["Intensity"] = []
        dict_r["Normalized intensity"] = []
        dict_r["Custom normalized intensity"] = []
        for s in ordered_sample_list:
            for r in range(df_to_plot_0.shape[0]):
                row = df_to_plot_0.iloc[r]
                if row.iloc[0] == s:
                    dict_r["Sample"].append(row.iloc[0])
                    dict_r["Intensity"].append(row.iloc[1])
                    dict_r["Normalized intensity"].append(row.iloc[2])
                    if len(norm_peak)>0:
                        dict_r["Custom normalized intensity"].append(row.iloc[3])
        
        df_to_plot = pd.DataFrame.from_dict(dict_r)
    else:
        df_to_plot = df_to_plot_0
    
    samples = []
    for r in range(df_to_plot.shape[0]):
        row = df_to_plot.iloc[r]
        sample_name = row.iloc[0]
        if row.iloc[0] not in samples:
            samples.append(sample_name)
            data_per_sample_N[sample_name] = []
            data_per_sample_CN[sample_name] = []

        data_per_sample_N[sample_name].append(row.iloc[2])
        if len(norm_peak)>0:
            data_per_sample_CN[sample_name].append(row.iloc[3])

    averages_N[peak] = []
    averages_CN[peak] = []
    for sample_name in data_per_sample_N:
        #averages_N[sample_name] = []
        ave = np.average(data_per_sample_N[sample_name])
        #averages_N[sample_name].append(ave) # Qui devo stare attento poi a legare bene il valore ave al picco 
        averages_N[peak].append(ave)
        #averages_CN[sample_name] = []
        ave_cn = np.average(data_per_sample_CN[sample_name])
        #averages_CN[sample_name].append(ave_cn) # Qui devo stare attento poi a legare bene il valore ave al picco
        averages_CN[peak].append(ave_cn)
    
    
    fig = px.box(df_to_plot, x="Sample", y="Normalized intensity", points="all", title=peak)
    fig.update_xaxes(tickangle=45)
    
    if peak in important_peaks:
        path_save = directory_res_tot
    else :
        path_save = directory_res_tot_extra
    
    csv_save = path_save + "\\" + peak + ".csv"
    df_to_plot.to_csv(csv_save, index=False)
    
    plot_save = path_save + "\\" + peak + ".png"
    fig.write_image(plot_save)
    
    #fig.show()
    
    #df_to_plot["Normalized intensity"] = df_to_plot["Normalized intensity"]/max(df_to_plot["Normalized intensity"])
    #fig = px.box(df_to_plot, x="Sample", y="Normalized intensity", points="all", title=peak)
    #fig.update_layout(yaxis_range=[0,1.05])
    #fig.update_xaxes(tickangle=45)
    
    #if peak in important_peaks:
    #    path_save = directory_res_tot
    #else :
    #    path_save = directory_res_tot_extra
    
    #csv_save = path_save + "\\Internally_normalized_" + peak + ".csv"
    #df_to_plot.to_csv(csv_save, index=False)
    
    #plot_save = path_save + "\\Internally_normalized_" + peak + ".png"
    #fig.write_image(plot_save)
    
    #fig.show()
    
ave_N_df = pd.DataFrame.from_dict(averages_N)
ave_N_df.index = samples
#averages_N["Sample"] = samples
csv_save = directory_res + "\\All_averages_normalized_total.csv"
ave_N_df.to_csv(csv_save, index=True)

if len(norm_peak)>0:
    ave_CN_df = pd.DataFrame.from_dict(averages_CN)
    ave_CN_df.index = samples
    #averages_CN["Sample"] = samples
    csv_save = directory_res + "\\All_averages_normalized_" + str(norm_peak) + ".csv"
    ave_CN_df.to_csv(csv_save, index=True)

In [ ]:
# Plotting for customized ions normalization
if len(norm_peak)>0:
    for peak in dict_sums:
        df_to_plot_0 = pd.DataFrame.from_dict(dict_sums[peak])
        if reordering == 1:
            dict_r = {}
            dict_r["Sample"] = []
            dict_r["Intensity"] = []
            dict_r["Normalized intensity"] = []
            dict_r["Custom normalized intensity"] = []
            for s in ordered_sample_list:
                for r in range(df_to_plot_0.shape[0]):
                    row = df_to_plot_0.iloc[r]
                    if row.iloc[0] == s:
                        dict_r["Sample"].append(row.iloc[0])
                        dict_r["Intensity"].append(row.iloc[1])
                        dict_r["Normalized intensity"].append(row.iloc[2])
                        dict_r["Custom normalized intensity"].append(row.iloc[3])

            df_to_plot = pd.DataFrame.from_dict(dict_r)
        else:
            df_to_plot = df_to_plot_0
        
        fig = px.box(df_to_plot, x="Sample", y="Custom normalized intensity", points="all", title=peak)
        fig.update_xaxes(tickangle=45)

        if peak in important_peaks:
            path_save = directory_res_cust
        else :
            path_save = directory_res_cust_extra

        csv_save = path_save + "\\" + peak + ".csv"
        df_to_plot.to_csv(csv_save, index=False)

        plot_save = path_save + "\\" + peak + ".png"
        fig.write_image(plot_save)

        #fig.show()

In [12]:
pkl_filename = path_data + "\\" + "dict_sums_images.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(dict_sums, file)

In [ ]:
pkl_filename = path_data + "\\" + "dict_all_images.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(dict_all, file)